In [1]:
import os, sys
sys.path.insert(0, '/clusterCAD')
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "clusterCAD.settings")
import django
django.setup()
import pks.models
import compounddb.models

In [2]:
# save compounds in the database
caf = compounddb.models.Compound(name="Caffeine", smiles="O=C2N(c1ncn(c1C(=O)N2C)C)C")
# caf.computeInchi()
caf.save()

cw = compounddb.models.Compound(name="cw", smiles="CCCC1=CC=C(C=C1)C(C)C(=O)O")
# cw.computeInchi()
cw.save()

In [3]:
caf.aMW()
caf.inchiKey
# caf.delete()

'RYYVLZVUVIJVGH-UHFFFAOYSA-N'

In [4]:
# test substructure search
compounds = compounddb.models.Compound.objects.raw('select \"inchiKey\" from compounddb_compound where m@>\'CCCc1ccc(C(C)C(=O)O)cc1\';')
[x.name for x in compounds]

['cw']

In [5]:
# query for top 10 hits to make a desired compound
queryString = 'CC(O)C(C)C(O)C(C)C(O)CC(O)CC(O)CC(O)CC(O)CC(O)CC(O)CC(O)CC(O)C(C)C(O)CC(=O)CC(O)CC(O)CC(O)CC(O)CC(=O)[S]'
results = compounddb.models.Compound.atomPairSearch(queryString, maxHits=10, minSim=0.1)
[(result[0], result[1].mol()) for result in results]

[(1.0, <rdkit.Chem.rdchem.Mol at 0x7f682380ae18>),
 (0.884758364312268, <rdkit.Chem.rdchem.Mol at 0x7f682380a868>),
 (0.870445344129555, <rdkit.Chem.rdchem.Mol at 0x7f682380ac10>),
 (0.864969562811289, <rdkit.Chem.rdchem.Mol at 0x7f682380af50>),
 (0.860752386299832, <rdkit.Chem.rdchem.Mol at 0x7f682380adb0>),
 (0.807939287799183, <rdkit.Chem.rdchem.Mol at 0x7f682380ad48>),
 (0.805355303810505, <rdkit.Chem.rdchem.Mol at 0x7f6822d84048>),
 (0.796536796536797, <rdkit.Chem.rdchem.Mol at 0x7f6822d840b0>),
 (0.762241292276628, <rdkit.Chem.rdchem.Mol at 0x7f6822d84118>),
 (0.761052631578947, <rdkit.Chem.rdchem.Mol at 0x7f6822d84180>)]

In [13]:
# find which gene clusters, subunit, and module each top hit is from
[[(module.subunit.cluster, module.subunit, module)
  for module in pks.models.Module.objects.filter(product=result[1])]
     for result in results]

def moduleOrder(self):
    pos = list(pks.models.Module.objects.filter(subunit__cluster = self.subunit.cluster).order_by('subunit__order', 'order')).index(self)
    return "Module " + str(pos + 1)
    
for result in results:
    print(result[0])
    for module in pks.models.Module.objects.filter(product=result[1]):
        print((module.subunit.cluster, module.subunit, moduleOrder(module)))

1.0
(<Cluster: Amphotericin pks gene cluster>, <Subunit: amphJ pks subunit>, 'Module 18')
(<Cluster: Nystatin pks gene cluster>, <Subunit: nysJ pks subunit>, 'Module 18')
(<Cluster: Nystatin-like Pseudonocardia polyene pks gene cluster>, <Subunit: cppJ pks subunit>, 'Module 18')
0.884758364312268
(<Cluster: Amphotericin pks gene cluster>, <Subunit: amphJ pks subunit>, 'Module 17')
(<Cluster: Nystatin pks gene cluster>, <Subunit: nysJ pks subunit>, 'Module 17')
(<Cluster: Nystatin-like Pseudonocardia polyene pks gene cluster>, <Subunit: cppJ pks subunit>, 'Module 17')
0.870445344129555
(<Cluster: Candicidin pks gene cluster>, <Subunit: fscE pks subunit>, 'Module 18')
0.864969562811289
(<Cluster: Candicidin pks gene cluster>, <Subunit: fscE pks subunit>, 'Module 19')
0.860752386299832
(<Cluster: Nystatin-like Pseudonocardia polyene pks gene cluster>, <Subunit: cppK pks subunit>, 'Module 19')
(<Cluster: Amphotericin pks gene cluster>, <Subunit: amphK pks subunit>, 'Module 19')
0.807939287

In [7]:
# find all cyclizing TEs and order by increasing MW of product
cyclicTEs = pks.models.TE.objects.filter(cyclic=True)
MWs = [(cyclicTE.module.subunit.cluster.description, cyclicTE.module.product.aMW()) for cyclicTE in cyclicTEs]
sortedTEs = sorted(MWs, key=lambda MW: MW[1])
sortedTEs

[('Simocyclinone', 244.287),
 ('Gulmirecin', 302.367),
 ('Methymycin / pikromycin', 386.485),
 ('Erythromycin', 402.528),
 ('Borrelidin', 446.537),
 ('Soraphen', 450.525),
 ('Tiacumicin B', 464.596),
 ('Thuggacin', 534.687),
 ('Spirangien', 756.968),
 ('Nystatin', 875.056)]